# Massaging to Improve Bias

### Imports and Creating the Dataframe

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, accuracy_score, classification_report, recall_score, f1_score
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult

# Import files for Fairness Metrics
from individual_fairness import eval_ind_fairness 
from disparate_impact import eval_disparate_impact
from counterfactual_fairness import evaluate_counterfactual_fairness_sex
from equality_fairness import eval_equality

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])
np.random.seed(42)

# Convert the Dataset into a Dataframe for easier massaging
df_original = pd.DataFrame(columns=dataset_orig.feature_names, data=dataset_orig.features)
df_original['Income Binary'] = dataset_orig.labels


### Helper Functions

In [119]:
def eval_performance(y_test, y_pred):
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Display metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

## Masaging to Remove Bias

In [120]:
def compute_discrimination(df, sensitive_attr, class_attr, privileged_value, positive_class):
    # Compute the discrimination score (difference in positive outcome rates).
    privileged = df[df[sensitive_attr] == privileged_value]
    unprivileged = df[df[sensitive_attr] != privileged_value]

    pos_rate_privileged = sum(privileged[class_attr] == positive_class) / len(privileged)
    pos_rate_unprivileged = sum(unprivileged[class_attr] == positive_class) / len(unprivileged)

    return pos_rate_privileged - pos_rate_unprivileged

def compute_m(df, sensitive_attr, class_attr, privileged_value, positive_class):
    # Compute number of instances M to relabel.
    disc = compute_discrimination(df, sensitive_attr, class_attr, privileged_value, positive_class)
    
    n_privileged = len(df[df[sensitive_attr] == privileged_value])
    n_unprivileged = len(df[df[sensitive_attr] != privileged_value])
    
    return int(disc * (n_privileged * n_unprivileged) / len(df))

def rank_instances(df, features, sensitive_attr, class_attr):
    # Train a classifier to rank instances by likelihood of being positive.
    X = df[features]
    y = df[class_attr]

    model = DecisionTreeClassifier()
    model.fit(X, y)
    
    scores = model.predict_proba(X)[:, 1]  # Probability of positive class
    df['score'] = scores
    return df

def apply_massaging(df, sensitive_attr, class_attr, privileged_value, positive_class):
    # Perform massaging technique.
    # Step 1: Compute M
    M = compute_m(df, sensitive_attr, class_attr, privileged_value, positive_class)
    print(f"Number of label changes (M): {M}")

    if M == 0:
        print("No massaging needed.")
        return df

    # Step 2: Rank instances
    features = [col for col in df.columns if col not in [sensitive_attr, class_attr]]
    df = rank_instances(df, features, sensitive_attr, class_attr)

    # Step 3: Modify labels
    unprivileged_neg = df[(df[sensitive_attr] != privileged_value) & (df[class_attr] != positive_class)]
    privileged_pos = df[(df[sensitive_attr] == privileged_value) & (df[class_attr] == positive_class)]

    # Promote top M from unprivileged_neg
    df.loc[unprivileged_neg.nlargest(M, 'score').index, class_attr] = positive_class

    # Demote bottom M from privileged_pos
    df.loc[privileged_pos.nsmallest(M, 'score').index, class_attr] = 1 - positive_class

    # Drop the ranking column
    df.drop(columns=['score'], inplace=True)

    return df


#### Baseline Model

In [121]:
def evaluate_counterfactual_fairness_sex(model, X):
    """
    Evaluates counterfactual fairness by flipping the 'sex' attribute.
    """
    # Store the original predictions on the test set
    original_predictions = model.predict(StandardScaler().fit_transform(X))

    # Create a copy of the dataset
    X_counterfactual = X.copy()

    # Flip 'sex' (0 -> 1, 1 -> 0)
    if 'sex' in X.columns:
        X_counterfactual['sex'] = 1 - X_counterfactual['sex']
    else:
        raise KeyError(f"The sensitive attribute 'sex' is not present in the dataset.")

    # Get counterfactual predictions
    counterfactual_predictions = model.predict(StandardScaler().fit_transform(X_counterfactual))

    # Compare the original and counterfactual predictions
    comparison = pd.DataFrame({
        'original': original_predictions,
        'counterfactual': counterfactual_predictions,
        'same_decision': original_predictions == counterfactual_predictions
    })

    # Return the comparison dataframe
    return comparison

In [125]:
scale_orig = StandardScaler()
lmod = LogisticRegression()

x_orig = df_original.drop(columns=['Income Binary'])
y_orig = df_original['Income Binary']

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_orig, y_orig, test_size=0.2, random_state=42, stratify=y_orig)

# Scale numerical features for training
x_train = scale_orig.fit_transform(x_train)

# Train the model
lmod.fit(x_train, y_train)

# Make predictions
x_test_scaled = scale_orig.fit_transform(x_test)
y_pred = lmod.predict(x_test_scaled)

# Evaluate performance of baseline model
eval_performance(y_test, y_pred)

# Evaluate Individual Fairness
ind_fairness_orig = eval_ind_fairness(x_train, y_train, x_test_scaled, y_pred)
print(f'Individual Fairness For Baseline: {ind_fairness_orig:.4f}')

# Evaluate Disparate Impact
disparate_impact_orig = eval_disparate_impact(x_test['sex'], y_pred)
print(f'Disparate Impact Score for Baseline: {disparate_impact_orig:.4f}')

# Evaluate Counterfactual Fairness
counterfactual_fairness = evaluate_counterfactual_fairness_sex(lmod, x_test)
fairness_metric_sex = counterfactual_fairness['same_decision'].mean()
print(f'Counterfactual Fairness Score For Baseline: {fairness_metric_sex:.4f}')


Accuracy: 0.8043
Precision: 0.6587
Recall: 0.3781
F1 Score: 0.4804

Classification Report:
               precision    recall  f1-score   support

         0.0       0.83      0.94      0.88      7431
         1.0       0.66      0.38      0.48      2338

    accuracy                           0.80      9769
   macro avg       0.74      0.66      0.68      9769
weighted avg       0.79      0.80      0.78      9769

Individual Fairness For Baseline: 0.7947
Disparate Impact Score for Baseline: 0.0000
Counterfactual Fairness Score For Baseline: 0.8063


In [127]:
sensitive_attribute_index = 1 # index of the 'sex' column in x_test_scaled
tpr_diff, eod = eval_equality(x_test_scaled, y_pred, sensitive_attribute_index, y_test)

print("Equality of Opportunity (EO):", tpr_diff)
print("Equality of Odds (EOd):", eod)


Equality of Opportunity (EO): 0.4471421345472939
Equality of Odds (EOd): 0.548178975491481


#### Model Performance After Massaging

In [128]:
# Apply massaging technique
df_massaged = apply_massaging(df_original, 'sex', 'Income Binary', privileged_value=1, positive_class=1)

# Repeat training and testing with massaged data
x_massaged = df_massaged.drop(columns=['Income Binary'])
y_massaged = df_massaged['Income Binary']

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_massaged, y_massaged, test_size=0.2, random_state=42, stratify=y_massaged)

# Scale numerical features for training
x_train = scale_orig.fit_transform(x_train)

# Train the model
lmod.fit(x_train, y_train)

# Make predictions
x_test_scaled = scale_orig.fit_transform(x_test)
y_pred = lmod.predict(x_test_scaled)

# Evaluate performance of baseline model
eval_performance(y_test, y_pred)

# Evaluate Individual Fairness
ind_fairness_massaged = eval_ind_fairness(x_train, y_train, x_test_scaled, y_pred)
print(f'Individual Fairness Ater Massaging: {ind_fairness_massaged:.4f}')

# # Evaluate Disparate Impact
disparate_impact_massaged = eval_disparate_impact(x_test['sex'], y_pred) # x_test[:,1] is the column for the 'sex' attribute
print(f'Disparate Impact Score After Massaging: {disparate_impact_massaged:.4f}')

# # Evaluate Counterfactual Fairness
counterfactual_fairness = evaluate_counterfactual_fairness_sex(lmod, x_test)
fairness_metric_sex = counterfactual_fairness['same_decision'].mean()
print(f'Counterfactual Fairness Score After Massaging: {fairness_metric_sex:.4f}')

Number of label changes (M): 2105
Accuracy: 0.8645
Precision: 0.7367
Recall: 0.6749
F1 Score: 0.7045

Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.92      0.91      7431
         1.0       0.74      0.67      0.70      2338

    accuracy                           0.86      9769
   macro avg       0.82      0.80      0.81      9769
weighted avg       0.86      0.86      0.86      9769

Individual Fairness Ater Massaging: 0.8892
Disparate Impact Score After Massaging: 0.8512
Counterfactual Fairness Score After Massaging: 0.9767


In [132]:
sensitive_attribute_index = 1  # Set the correct index of the 'sex' column in x_test_scaled
tpr_diff, eod = eval_equality(x_test_scaled, y_pred, sensitive_attribute_index, y_test)

print("Equality of Opportunity (EO) After Massaging:", tpr_diff)
print("Equality of Odds (EOd) After Massaging:", eod)


Equality of Opportunity (EO) After Massaging: 0.17111508452535762
Equality of Odds (EOd) After Massaging: 0.27554806422382366


## Comments

This version of the massaging technique uses the same preprocessing as adult_reweighing.ipynb (from AIF360) for the baseline model. The baseline model's performance is nearly identical to that found in adult_reweighing.ipynb. 

## Results

Massaging the data improved the model's predictive performance. This could be because the baseline model relies to heavily on the "sex" attribute. The model's individual fairness increased significantly after massaging. The disparate impact and counterfactual fairness of the model increased with massaging. The Equality of Opportuniy and Equality of Odds are lowered after massaging indicating the model is fair in predicting both positive and negative outcomes for both sexes.